# Feature_Compare.ipynb

This notebook allows you to extract features from time-domain and frequency-domain waveforms. The features were designed to characterize the ovserall shape of the waveforms. The end of the notebook creates a pairplot for the extracted features for a sample of waveforms created by varying a single user-specified parameter. Images are saved to the `./images/` directory.

Some of the functions used to extract features are inefficient and need to be optimized. Using this notebook to generate and plot a sample of 100 waveforms takes O(5 min), so if we want to do something like this on a larger dataset we will have to spend some time removing for-loops.

In [ ]:
# Standard imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys

# PyCBC imports
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.waveform import td_approximants, fd_approximants
from pycbc.detector import Detector
from pycbc.filter import highpass
from pycbc.catalog import Merger
from pycbc.conversions import mass1_from_mchirp_q

# Magic
%matplotlib inline

## Simulation of pure waveforms for many parameters

Generate a random frequency-domain waveform if desired

In [ ]:
masses = np.linspace(1.1, 2.0, 11)
spins = np.linspace(-1., 1., 11)
eccentricities = np.linspace(0., 1., 11)
tidal_deformabilities = np.linspace(0., 1., 11) #Check value range
quadrupole_monopoles = np.linspace(0., 1., 11) #Check value range
octupolar_tidal_deformabilities = np.linspace(0., 1., 11) #Check value range
quadrupolar_frequencies = np.linspace(1., 100., 11) #Check value range
octupolar_frequencies = np.linspace(1., 100., 11) #Check value range
distances = np.linspace(1., 200., 200)
coalescence_phases = np.linspace(0., 2 * np.pi, 100)
inclinations = np.linspace(0.0, np.pi / 2, 100)
ascending_node_axis_longitudes = np.linspace(0., 2 * np.pi, 100)
mean_anomalies = np.linspace(0., 2 * np.pi, 100)


kwargs = [{'mass1': np.random.choice(masses, 1)[0],
           'mass2': np.random.choice(masses, 1)[0],
           'spin1z': np.random.choice(spins, 1)[0],
           'spin2z': np.random.choice(spins, 1)[0],
           'eccentricity': np.random.choice(eccentricities, 1)[0],
           'lambda1': np.random.choice(tidal_deformabilities, 1)[0],
           'lambda2': np.random.choice(tidal_deformabilities, 1)[0],
           'dquad_mon1': np.random.choice(quadrupole_monopoles, 1)[0],
           'dquad_mon2': np.random.choice(quadrupole_monopoles, 1)[0],
           'lambda_octu1': np.random.choice(octupolar_tidal_deformabilities, 1)[0],
           'lambda_octu2': np.random.choice(octupolar_tidal_deformabilities, 1)[0],
           'quadfmode1': np.random.choice(quadrupolar_frequencies, 1)[0],
           'quadfmode2': np.random.choice(quadrupolar_frequencies, 1)[0],
           'octufmode1': np.random.choice(octupolar_frequencies, 1)[0],
           'octufmode2': np.random.choice(octupolar_frequencies, 1)[0],
           'distance': np.random.choice(distances, 1)[0],
           'coa_phase': np.random.choice(coalescence_phases, 1)[0],
           'inclination': np.random.choice(inclinations, 1)[0],
           'long_asc_nodes': np.random.choice(ascending_node_axis_longitudes, 1)[0],
           'mean_per_ano': np.random.choice(mean_anomalies, 1)[0],
           'delta_f': 1./4096,
           'f_lower': 20.,
           'approximant': "TaylorF2", 
           'f_ref': 0
          } for _ in range(30)]

## Functions to calculate features

In [ ]:
# Combine time and frequency features and load into dataframe
def get_features(hp_t, hp_f):
    cols = ['t_1', 't_2', 't_3', 't_4', 't_5', 'f_1', 'f_2', 'f_3', 'f_4', 'f_5']
    data = get_features_t(hp_t) + get_features_f(hp_f)
    return pd.DataFrame(data=[data], columns=cols)

In [ ]:
# Extract features from time-domain waveform
def get_features_t(hp):
    data = []
    
    #Downselect window of waveform to be only 100 sec before merger
    strains = hp.data[hp.sample_times > -100.0]
    times = hp.sample_times[hp.sample_times > -100.0]
    
    #t_1 : amplitude, expected to directly relate to inclination
    amp = np.max(strains)
    data.append(amp)
    
    #t_2 : number of peaks above 0.25 * max amplitude to describe overal signal strength
    data.append(count_peaks(strains[strains > 0.25 * amp]))
    
    #t_3 : number of peaks after max to describe ringdown behavior
    time_of_max_amplitude = times[strains == amp][0]
    data.append(count_peaks(strains[times > time_of_max_amplitude]))
    
    #t_4 : max amp / median amp to describe total rise
    data.append(amp / np.median(strains))
    
    #t_5 : mean amp of normalized waveform to describe skew
    data.append(np.mean(strains / amp))
    
    return data

In [ ]:
# Extract features from frequency-domain waveform
def get_features_f(hp):
    data = []
    
    #Downselect window
    strains = np.real(hp.data[hp.sample_frequencies < 1000.0])
    frequencies = hp.sample_frequencies[hp.sample_frequencies < 1000.0]
    
    #t_1 : amplitude, expected to directly relate to inclination
    amp = np.max(strains)
    data.append(amp)
    
    #t_2 : number of peaks above 0.25 * max amplitude to describe overal signal strength
    data.append(count_peaks(strains[strains > 0.25 * amp]))
    
    #t_3 : number of peaks after max to describe ringdown behavior
    freq_of_max_amplitude = frequencies[strains == amp][0]
    data.append(count_peaks(strains[frequencies > freq_of_max_amplitude]))
    
    #t_4 : max amp / median amp to describe total rise
    if np.median(strains) != 0.0:
        data.append(amp / np.median(strains))
    else:
        data.append(1.e8)
    
    #t_5 : mean amp of normalized waveform to describe skew
    data.append(np.mean(strains / amp))
    
    return data

In [ ]:
# Helper functions for feature extraction
### These are inefficienct and need optimization

def count_peaks(strains):
    counter = 0
    for ii in range(len(strains) - 2):
        if strains[ii] < strains[ii+1] and strains[ii+1] > strains[ii+2]:
            counter += 1
    return counter

def find_first_peak_amp(strains):
    for ii in range(len(strains)):
        if strain[ii] > strain[ii+1]:
            return strain[ii]

## Baseline time and frequency parameter settings

In [ ]:
t_kwargs = {'mass1': 1.4,
           'mass2': 1.4,
           'spin1z': 1.0,
           'spin2z': 1.0,
           'eccentricity': 0.5,
           'lambda1': 0.5,
           'lambda2': 0.7,
           'dquad_mon1': 0.5,
           'dquad_mon2': 0.5,
           'lambda_octu1': 0.3,
           'lambda_octu2': 0.2,
           'quadfmode1': 50.,
           'quadfmode2': 45.,
           'octufmode1': 100.,
           'octufmode2': 70.,
           'distance': 90.,
           'coa_phase': 3.14,
           'inclination': np.pi / 2.,
           'long_asc_nodes': 0.3,
           'mean_per_ano': 4.0,
           'delta_t': 1./4096,
           'f_lower': 20.,
           'approximant': "TaylorF2", 
           'f_ref': 0}

f_kwargs = {'mass1': 1.4,
           'mass2': 1.4,
           'spin1z': 1.0,
           'spin2z': 1.0,
           'eccentricity': 0.5,
           'lambda1': 0.5,
           'lambda2': 0.7,
           'dquad_mon1': 0.5,
           'dquad_mon2': 0.5,
           'lambda_octu1': 0.3,
           'lambda_octu2': 0.2,
           'quadfmode1': 50.,
           'quadfmode2': 45.,
           'octufmode1': 100.,
           'octufmode2': 70.,
           'distance': 90.,
           'coa_phase': 3.14,
           'inclination': np.pi / 2.,
           'long_asc_nodes': 0.3,
           'mean_per_ano': 4.0,
           'delta_f': 1./4096,
           'f_lower': 20.,
           'approximant': "TaylorF2", 
           'f_ref': 0}

## Step through a setting and evaluate waveform features

In [ ]:
setting_name = 'coa_phase'
setting_min = 0.
setting_max = 2 * np.pi
num_steps = 100
setting_range = np.linspace(setting_min, setting_max, num_steps)

########################################################

dfs = []
counter = 0
for setting in setting_range:
    #Track progress
    counter += 1
    sys.stdout.write('\r%i / %i' %(counter, num_steps))
    sys.stdout.flush()
    
    #Copy baseline dictionaries
    t_kwargs_copy = t_kwargs.copy()
    f_kwargs_copy = f_kwargs.copy()
    
    #Set the new setting value
    t_kwargs_copy[setting_name] = setting
    f_kwargs_copy[setting_name] = setting
    
    #Generate waveforms
    hp_t, hc_t = get_td_waveform(**t_kwargs_copy)
    hp_f, hc_f = get_fd_waveform(**f_kwargs_copy)
    
    #Calculate features
    df = get_features(hp_t, hp_f)
    dfs.append(df)

df = pd.concat(dfs)

Compare features and save pairplot to images directory

In [ ]:
sns.pairplot(df)
plt.savefig('images/%s.png' %setting_name)
plt.show()